In [3]:
using CSV
using DataFrames
using CategoricalArrays

# Load the dataset from a CSV file
df = CSV.read("demo_clusters.csv", DataFrame)

# Ensure 'Cluster' is treated as a categorical variable (if it's not already)
df.Cluster = categorical(df.Cluster)

# Create a GridSearch object for hyperparameter tuning
grid = IAI.GridSearch(
    IAI.OptimalTreeClassifier(
        max_depth=4,               # Maximum depth of the tree
        criterion=:misclassification,  # Split criterion for the classifier
        missingdatamode=:separate_class  # Handle missing data as a separate class
    )
)

# Select all columns except the last column for features (assuming 'Cluster' is the target)
df_selected = df[:, 4:end-1]

# Fit the model, using the features (all columns except 'Cluster') and the target ('Cluster')
IAI.fit!(grid, df_selected, df.Cluster)

# Evaluate the classification accuracy
accuracy = IAI.score(grid, df_selected, df.Cluster)
println("Classification Accuracy: ", accuracy)

# Visualize the tree
IAI.show_in_browser(grid)



Classification Accuracy: 0.9650190114068441


"/var/folders/p_/j5379bnd7t7f3l97gslk8q_80000gn/T/jl_0SLsnf.html"

In [4]:
using CSV
using DataFrames
using CategoricalArrays

# Load the dataset from a CSV file
df = CSV.read("demo_clusters.csv", DataFrame)

# Ensure 'Cluster' is treated as a categorical variable (if it's not already)
df.Cluster = categorical(df.Cluster)

# Create a GridSearch object for hyperparameter tuning
grid = IAI.GridSearch(
    IAI.OptimalTreeClassifier(
        max_depth=5,               # Maximum depth of the tree
        criterion=:misclassification,  # Split criterion for the classifier
        missingdatamode=:separate_class  # Handle missing data as a separate class
    )
)

# Select all columns except the last column for features (assuming 'Cluster' is the target)
df_selected = df[:, 4:end-1]

# Fit the model, using the features (all columns except 'Cluster') and the target ('Cluster')
IAI.fit!(grid, df_selected, df.Cluster)

# Evaluate the classification accuracy
accuracy = IAI.score(grid, df_selected, df.Cluster)
println("Classification Accuracy: ", accuracy)

# Visualize the tree
IAI.show_in_browser(grid)

Classification Accuracy: 0.9657794676806084


"/var/folders/p_/j5379bnd7t7f3l97gslk8q_80000gn/T/jl_vN1b3a.html"

In [5]:
using CSV
using DataFrames
using CategoricalArrays

# Load the dataset from a CSV file
df = CSV.read("demo_clusters.csv", DataFrame)

# Ensure 'Cluster' is treated as a categorical variable (if it's not already)
df.Cluster = categorical(df.Cluster)

# Create a GridSearch object for hyperparameter tuning
grid = IAI.GridSearch(
    IAI.OptimalTreeClassifier(
        max_depth=3,               # Maximum depth of the tree
        criterion=:misclassification,  # Split criterion for the classifier
        missingdatamode=:separate_class  # Handle missing data as a separate class
    )
)

# Select all columns except the last column for features (assuming 'Cluster' is the target)
df_selected = df[:, 4:end-1]

# Fit the model, using the features (all columns except 'Cluster') and the target ('Cluster')
IAI.fit!(grid, df_selected, df.Cluster)

# Evaluate the classification accuracy
accuracy = IAI.score(grid, df_selected, df.Cluster)
println("Classification Accuracy: ", accuracy)

# Visualize the tree
IAI.show_in_browser(grid)

Classification Accuracy: 0.9277566539923955


"/var/folders/p_/j5379bnd7t7f3l97gslk8q_80000gn/T/jl_4hcv6J.html"